In [3]:
import csv
import re

def load_dataset(filename):
    dataset = {}
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for row in reader:
            number = int(row[0])
            meru_number = row[1]
            grammar = row[2]
            morphemes = re.findall(r'\b\w+\b', meru_number)
            dataset[number] = (meru_number, grammar, morphemes)
    return dataset

def analyze_meru_number(input_number, dataset):
    if input_number in dataset:
        # Number is in the dataset
        return dataset[input_number]

    if 100 <= input_number < 200:
        tens = (input_number % 100) // 10
        ones = input_number % 10
        if tens == 0:
            meru_number = f"{dataset[100][0]} {dataset[ones][0]}"
            grammar = f"{dataset[100][1]} {dataset[ones][1]}"
        else:
            meru_number = f"{dataset[100][0]} {dataset[tens * 10][0]} na {dataset[ones][0]}"
            grammar = f"{dataset[100][1]} {dataset[tens * 10][1]} na {dataset[ones][1]}"
        return meru_number, grammar, dataset[100][2] + dataset[ones][2]

    if input_number < 100:
        # Numbers from 1 to 99
        tens = input_number // 10
        ones = input_number % 10
        meru_number = f"{dataset[tens * 10][0]} {dataset[ones][0]}"
        grammar = f"{dataset[tens * 10][1]} {dataset[ones][1]}"
        return meru_number, grammar, dataset[tens * 10][2] + dataset[ones][2]

    if input_number >= 200:
        hundreds = input_number // 100
        remainder = input_number % 100
        if remainder == 0:
            meru_number = f"{dataset[hundreds][0]} {dataset[100][0]}"
            grammar = f"{dataset[hundreds][1]} {dataset[100][1]}"
        else:
            tens = remainder // 10
            ones = remainder % 10
            if tens == 0:
                meru_number = f"{dataset[hundreds][0]} {dataset[100][0]} {dataset[ones][0]}"
                grammar = f"{dataset[hundreds][1]} {dataset[100][1]} {dataset[ones][1]}"
            else:
                meru_number = f"{dataset[hundreds][0]} {dataset[100][0]} {dataset[tens * 10][0]} na {dataset[ones][0]}"
                grammar = f"{dataset[hundreds][1]} {dataset[100][1]} {dataset[tens * 10][1]} na {dataset[ones][1]}"
        return meru_number, grammar, dataset[hundreds][2] + dataset[100][2] + dataset[tens * 10][2] + dataset[ones][2]

def run_meru_number_analyzer():
    dataset = load_dataset('MeruNumbers.csv')
    while True:
        print("Options:")
        print("1. Number to Meru conversion")
        print("2. Meru to Number conversion")
        print("3. Quit")
        choice = input("Enter your choice (1-3): ")

        if choice == '1':
            # Number to Meru conversion
            input_number = int(input("Enter a number: "))
            meru_number, grammar, morphemes = analyze_meru_number(input_number, dataset)
            print(f"The Meru number for {input_number} is: {meru_number} ({grammar})")
            print(f"Morphemes: {', '.join(morphemes)}")
            print()
        elif choice == '2':
            # Meru to Number conversion
            input_meru = input("Enter a Meru number: ")
            found_number = False
            for number, (meru, _, _) in dataset.items():
                if input_meru == meru:
                    print(f"The number equivalent of {input_meru} is: {number}")
                    found_number = True
                    break
            if not found_number:
                print("Meru number not found in the dataset.")
            print()
        elif choice == '3':
            # Quit the program
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please enter a valid option (1-3).")
            print()

run_meru_number_analyzer()


Options:
1. Number to Meru conversion
2. Meru to Number conversion
3. Quit


The Meru number for 1 is:  imwe ( i-mwe)
Morphemes: imwe

Options:
1. Number to Meru conversion
2. Meru to Number conversion
3. Quit
The Meru number for 10 is:  ikumi ( i-kumi)
Morphemes: ikumi

Options:
1. Number to Meru conversion
2. Meru to Number conversion
3. Quit
The Meru number for 55 is: mirongo itano n aithano (mi-rongo-i-tano na i-thano)
Morphemes: mirongo, itano, n, aithano

Options:
1. Number to Meru conversion
2. Meru to Number conversion
3. Quit
Meru number not found in the dataset.

Options:
1. Number to Meru conversion
2. Meru to Number conversion
3. Quit
Meru number not found in the dataset.

Options:
1. Number to Meru conversion
2. Meru to Number conversion
3. Quit
The number equivalent of mirongoiri is: 20

Options:
1. Number to Meru conversion
2. Meru to Number conversion
3. Quit
The number equivalent of igana is: 100

Options:
1. Number to Meru conversion
2. Meru to Number conversion
3. Quit
Exiting...
